In [1]:
from dl.simple_cnn import SimpleCNN
from dl.shuffle_net import ShuffleNetV2
from dl.efficient_net import EfficientNet
from dl.callbacks.plot_loss import PlotLosses
from data.data_set import get_data
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, TerminateOnNaN, EarlyStopping, ModelCheckpoint, TensorBoard
from config import DL_MODELS_PATH, TB_LOGS_PATH
import os
import matplotlib.pyplot as plt
import numpy as np
import pathlib
from swa.tfkeras import SWA
from dl.callbacks.cyclical_lr import CyclicLR
from plotly.offline import init_notebook_mode
from visualization.visualize_history import plot_accuracy_and_loss
init_notebook_mode(connected=True)

2019-12-16 14:58:00,412 : INFO : Checking/creating directories...
2019-12-16 14:58:00,413 : INFO : Directories are set.


In [2]:
((X_train, y_train), (X_val, y_val)) = get_data()

In [3]:
data_generator = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range= .8,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = .1, # Randomly zoom image 
        width_shift_range= .1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range= .1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=.05,  # randomly flip images horizontally
        vertical_flip=False)  # randomly flip images vertically

In [4]:
def get_callbacks(model_name, use_cyclic_lr=False, plot=False):
    callbacks = [TerminateOnNaN(), EarlyStopping(monitor='val_accuracy', patience=5)]
    callbacks.append(ModelCheckpoint(os.path.join(DL_MODELS_PATH, model_name + '-{epoch:02d}-{val_accuracy:.2f}.hdf5'), monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max'))
    tb_path = os.path.join(TB_LOGS_PATH, model_name)
    pathlib.Path(tb_path).mkdir(parents=True, exist_ok=True)
    callbacks.append(TensorBoard(log_dir=tb_path))
    callbacks.append(SWA(start_epoch=5, lr_schedule='constant', swa_lr=0.001, verbose=1))
    if plot:
        callbacks.append(PlotLosses())
    if use_cyclic_lr:
        callbacks.append(CyclicLR(base_lr=0.001, max_lr=0.006, step_size=2000.))
    else:
        rlronp =  ReduceLROnPlateau(monitor='val_loss',
                                    patience=3,
                                    verbose=1,
                                    factor=0.5,
                                    min_lr=0.00005)
        
        callbacks.append(rlronp)
    return callbacks

In [5]:
epochs = 100
Learning_rate = 0.001
decay= 5 * Learning_rate / epochs
optimizers = {'rmsprop': RMSprop(lr=Learning_rate, rho=0.9, epsilon=1e-08, decay= 0),
              'adam': Adam(lr=Learning_rate, decay= 3 * Learning_rate / epochs)}

In [6]:
histories = {}
epochs = 50
for opt_name, opt in optimizers.items():
    # Train a basic cnn
    model_name = f'simple_cnn_{opt_name}'
    model = SimpleCNN(opt)
    history = model.train(data_generator, X_train, y_train, X_val, y_val,
                          2048, epochs, get_callbacks(model_name))
    histories[model_name] = history.history
    
    # Train shufflenetv2
    model_name = f'shuffle_netv2_{opt_name}'
    model = SimpleCNN(opt)
    history = model.train(data_generator, X_train, y_train, X_val, y_val,
                          256, epochs, get_callbacks(model_name))
    histories[model_name] = history.history

Epoch 1/50
29/29 [============================>.] - ETA: 0s - loss: 1.0381 - accuracy: 0.6153 - categorical_crossentropy: 1.0381 - categorical_accuracy: 0.6153
Epoch 00001: val_categorical_accuracy improved from -inf to 0.58430, saving model to /home/cenk/Research/pre-trained-models/cv/fashion_mnist/simple_cnn_rmsprop-01-0.58.hdf5
30/29 [==============================] - 13s 448ms/step - loss: 1.0296 - accuracy: 0.6161 - categorical_crossentropy: 1.0354 - categorical_accuracy: 0.6161 - val_loss: 1.7119 - val_accuracy: 0.5843 - val_categorical_crossentropy: 1.7120 - val_categorical_accuracy: 0.5843
Epoch 2/50
29/29 [============================>.] - ETA: 0s - loss: 0.6526 - accuracy: 0.7555 - categorical_crossentropy: 0.6519 - categorical_accuracy: 0.7555
Epoch 00002: val_categorical_accuracy improved from 0.58430 to 0.76420, saving model to /home/cenk/Research/pre-trained-models/cv/fashion_mnist/simple_cnn_rmsprop-02-0.76.hdf5
30/29 [==============================] - 11s 357ms/step - l

Epoch 16/50
29/29 [============================>.] - ETA: 0s - loss: 0.2937 - accuracy: 0.8886 - categorical_crossentropy: 0.2941 - categorical_accuracy: 0.8886
Epoch 00016: val_categorical_accuracy improved from 0.89650 to 0.90460, saving model to /home/cenk/Research/pre-trained-models/cv/fashion_mnist/simple_cnn_rmsprop-16-0.90.hdf5
30/29 [==============================] - 11s 378ms/step - loss: 0.2940 - accuracy: 0.8887 - categorical_crossentropy: 0.2943 - categorical_accuracy: 0.8887 - val_loss: 0.2531 - val_accuracy: 0.9046 - val_categorical_crossentropy: 0.2531 - val_categorical_accuracy: 0.9046
Epoch 17/50
29/29 [============================>.] - ETA: 0s - loss: 0.2872 - accuracy: 0.8919 - categorical_crossentropy: 0.2875 - categorical_accuracy: 0.8919
Epoch 00017: val_categorical_accuracy did not improve from 0.90460
30/29 [==============================] - 11s 381ms/step - loss: 0.2871 - accuracy: 0.8921 - categorical_crossentropy: 0.2874 - categorical_accuracy: 0.8921 - val_l

Epoch 31/50
29/29 [============================>.] - ETA: 0s - loss: 0.2357 - accuracy: 0.9107 - categorical_crossentropy: 0.2364 - categorical_accuracy: 0.9107
Epoch 00031: val_categorical_accuracy did not improve from 0.92070

Epoch 00031: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
30/29 [==============================] - 11s 375ms/step - loss: 0.2364 - accuracy: 0.9106 - categorical_crossentropy: 0.2370 - categorical_accuracy: 0.9106 - val_loss: 0.2272 - val_accuracy: 0.9154 - val_categorical_crossentropy: 0.2270 - val_categorical_accuracy: 0.9154
Epoch 32/50
29/29 [============================>.] - ETA: 0s - loss: 0.2376 - accuracy: 0.9118 - categorical_crossentropy: 0.2368 - categorical_accuracy: 0.9118
Epoch 00032: val_categorical_accuracy did not improve from 0.92070
30/29 [==============================] - 11s 370ms/step - loss: 0.2366 - accuracy: 0.9121 - categorical_crossentropy: 0.2358 - categorical_accuracy: 0.9121 - val_loss: 0.2572 - val_accuracy: 

234/234 [============================>.] - ETA: 0s - loss: 0.2629 - accuracy: 0.9028 - categorical_crossentropy: 0.2627 - categorical_accuracy: 0.9028
Epoch 00012: val_categorical_accuracy did not improve from 0.91850
235/234 [==============================] - 18s 77ms/step - loss: 0.2628 - accuracy: 0.9028 - categorical_crossentropy: 0.2626 - categorical_accuracy: 0.9028 - val_loss: 0.2282 - val_accuracy: 0.9144 - val_categorical_crossentropy: 0.2273 - val_categorical_accuracy: 0.9144
Epoch 13/50
234/234 [============================>.] - ETA: 0s - loss: 0.2580 - accuracy: 0.9038 - categorical_crossentropy: 0.2578 - categorical_accuracy: 0.9038
Epoch 00013: val_categorical_accuracy did not improve from 0.91850
235/234 [==============================] - 18s 77ms/step - loss: 0.2578 - accuracy: 0.9039 - categorical_crossentropy: 0.2576 - categorical_accuracy: 0.9039 - val_loss: 0.2265 - val_accuracy: 0.9135 - val_categorical_crossentropy: 0.2270 - val_categorical_accuracy: 0.9135
Epoch 

29/29 [============================>.] - ETA: 0s - loss: 0.4986 - accuracy: 0.8124 - categorical_crossentropy: 0.4980 - categorical_accuracy: 0.8124
Epoch 00003: val_categorical_accuracy improved from 0.75750 to 0.78130, saving model to /home/cenk/Research/pre-trained-models/cv/fashion_mnist/simple_cnn_adam-03-0.78.hdf5
30/29 [==============================] - 11s 375ms/step - loss: 0.4975 - accuracy: 0.8127 - categorical_crossentropy: 0.4969 - categorical_accuracy: 0.8127 - val_loss: 1.3548 - val_accuracy: 0.7813 - val_categorical_crossentropy: 1.3549 - val_categorical_accuracy: 0.7813
Epoch 4/50
29/29 [============================>.] - ETA: 0s - loss: 0.4492 - accuracy: 0.8320 - categorical_crossentropy: 0.4494 - categorical_accuracy: 0.8320
Epoch 00004: val_categorical_accuracy did not improve from 0.78130
30/29 [==============================] - 11s 365ms/step - loss: 0.4483 - accuracy: 0.8324 - categorical_crossentropy: 0.4485 - categorical_accuracy: 0.8324 - val_loss: 1.2838 - va

29/29 [============================>.] - ETA: 0s - loss: 0.2771 - accuracy: 0.8947 - categorical_crossentropy: 0.2778 - categorical_accuracy: 0.8947
Epoch 00017: val_categorical_accuracy improved from 0.89800 to 0.90690, saving model to /home/cenk/Research/pre-trained-models/cv/fashion_mnist/simple_cnn_adam-17-0.91.hdf5
30/29 [==============================] - 11s 369ms/step - loss: 0.2772 - accuracy: 0.8947 - categorical_crossentropy: 0.2778 - categorical_accuracy: 0.8947 - val_loss: 0.2621 - val_accuracy: 0.9069 - val_categorical_crossentropy: 0.2622 - val_categorical_accuracy: 0.9069
Epoch 18/50
29/29 [============================>.] - ETA: 0s - loss: 0.2683 - accuracy: 0.8996 - categorical_crossentropy: 0.2683 - categorical_accuracy: 0.8996
Epoch 00018: val_categorical_accuracy did not improve from 0.90690
30/29 [==============================] - 11s 366ms/step - loss: 0.2681 - accuracy: 0.8998 - categorical_crossentropy: 0.2681 - categorical_accuracy: 0.8998 - val_loss: 0.2621 - v

Epoch 3/50
234/234 [============================>.] - ETA: 0s - loss: 0.3911 - accuracy: 0.8535 - categorical_crossentropy: 0.3914 - categorical_accuracy: 0.8535
Epoch 00003: val_categorical_accuracy improved from 0.86520 to 0.87420, saving model to /home/cenk/Research/pre-trained-models/cv/fashion_mnist/shuffle_netv2_adam-03-0.87.hdf5
235/234 [==============================] - 16s 69ms/step - loss: 0.3912 - accuracy: 0.8535 - categorical_crossentropy: 0.3914 - categorical_accuracy: 0.8535 - val_loss: 0.3376 - val_accuracy: 0.8742 - val_categorical_crossentropy: 0.3401 - val_categorical_accuracy: 0.8742
Epoch 4/50
234/234 [============================>.] - ETA: 0s - loss: 0.3572 - accuracy: 0.8651 - categorical_crossentropy: 0.3571 - categorical_accuracy: 0.8651
Epoch 00004: val_categorical_accuracy improved from 0.87420 to 0.88960, saving model to /home/cenk/Research/pre-trained-models/cv/fashion_mnist/shuffle_netv2_adam-04-0.89.hdf5
235/234 [==============================] - 16s 69ms

234/234 [============================>.] - ETA: 0s - loss: 0.2526 - accuracy: 0.9043 - categorical_crossentropy: 0.2527 - categorical_accuracy: 0.9043
Epoch 00017: val_categorical_accuracy did not improve from 0.91580
235/234 [==============================] - 16s 69ms/step - loss: 0.2526 - accuracy: 0.9044 - categorical_crossentropy: 0.2526 - categorical_accuracy: 0.9044 - val_loss: 0.2327 - val_accuracy: 0.9132 - val_categorical_crossentropy: 0.2353 - val_categorical_accuracy: 0.9132
Epoch 18/50
234/234 [============================>.] - ETA: 0s - loss: 0.2518 - accuracy: 0.9059 - categorical_crossentropy: 0.2517 - categorical_accuracy: 0.9059
Epoch 00018: val_categorical_accuracy improved from 0.91580 to 0.91990, saving model to /home/cenk/Research/pre-trained-models/cv/fashion_mnist/shuffle_netv2_adam-18-0.92.hdf5
235/234 [==============================] - 17s 70ms/step - loss: 0.2517 - accuracy: 0.9060 - categorical_crossentropy: 0.2515 - categorical_accuracy: 0.9060 - val_loss: 0

In [7]:
plot_accuracy_and_loss(histories)